This is the final notebook. Please keep it clean as it will be shared with the other team members. 

Please keep the code as clean as possible, when having it in the final notebook as other players might use it too.

Also, remember to take care of good commenting of the code as it will be shared with your coplayers later on.

# Importing

In [16]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from scipy.misc import toimage
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline  
from PIL import Image
import glob
import numpy as np
import pandas as pd
from IPython.display import Image
from IPython.display import Image
from IPython.display import Image
from IPython.core.display import HTML 
from multi_gpu import make_parallel

# Read the data

Read the file names from `../Data/Kaggle/train/*`. 

In [4]:
# the folder path to the folder with all the fishies in there
folders_path = '../Team 3/Zoomed train/*'

# creating a dataframe with the columns below
image_dict = {}

# going through all folders and saving file names in dataframe so that order is always correct
for foldername in glob.glob(folders_path):
    for image_name in glob.glob(foldername + '/*.jpg'):
        # getting image name and fish type and putting it in a dictionary
        fish_type = foldername[-3:]
        image_dict[image_name] = fish_type

image_df = pd.DataFrame(image_dict.items(), columns = ['image_name', 'fish_type']).sample(frac=1) # random sampling
dummy_df = pd.get_dummies(image_df, columns=['fish_type']) # dummify data
print(dummy_df)

                                      image_name  fish_type_ALB  \
1524    ../Team 3/Zoomed train/YFT/img_06567.jpg              0   
970     ../Team 3/Zoomed train/NoF/img_03423.jpg              0   
453     ../Team 3/Zoomed train/YFT/img_00969.jpg              0   
1988    ../Team 3/Zoomed train/NoF/img_02751.jpg              0   
1277    ../Team 3/Zoomed train/ALB/img_04890.jpg              1   
1432    ../Team 3/Zoomed train/ALB/img_00998.jpg              1   
9       ../Team 3/Zoomed train/ALB/img_01374.jpg              1   
2257    ../Team 3/Zoomed train/ALB/img_01033.jpg              1   
546     ../Team 3/Zoomed train/ALB/img_05846.jpg              1   
2237    ../Team 3/Zoomed train/NoF/img_07379.jpg              0   
1560    ../Team 3/Zoomed train/ALB/img_00039.jpg              1   
3515    ../Team 3/Zoomed train/YFT/img_02748.jpg              0   
3294    ../Team 3/Zoomed train/BET/img_01440.jpg              0   
1958    ../Team 3/Zoomed train/NoF/img_06256.jpg              

In [5]:
# Count of the image types
#image_df.groupby('fish_type').count().sort_values('image_name')

# Preprocess

# Manual preprocessing
* Read the image from file
* Resize it to (224, 224)
* Set type to np.float (such that it can be reschaled to 0-1)
* Rescale to 0-1
* Subtract mean

In [6]:
images = []

for filename in dummy_df.image_name:
    img = cv2.imread(filename) # Read file    
    img = cv2.resize(img, (224, 224)) # Resize image to 224 x 224 pixels (dont feel obliged to use this)
    img = img.astype(np.float32) # Change to 32 bit floats (good for neural networks, other floats not as much making sense)

    # "Normalize" images
    for rgb_value in [0,1,2]:
        img[:, :, rgb_value] = img[:, :, rgb_value] / 255.0 # 255 is the RGB maximum
        img[:, :, rgb_value] -= numpy.mean(img[:, :, rgb_value])

    img = img.transpose((2,0,1))
    images.append(img)

# X and y value and labels
x = np.asarray(images)
y = dummy_df.iloc[:,1:9].as_matrix()

# For inspection: table, just to be sure that everything goes well
labels = dummy_df.image_name.apply(lambda path: path[-13:])
dummy_df['image_bytes'] = images

split_no = int(len(x) * 0.9)

x_train = x[:split_no]
y_train = y[:split_no]
labels_train = labels[:split_no]

x_test = x[split_no:]
y_test = y[split_no:]
labels_test = labels[split_no:]
print(labels_test)

2352    img_01756.jpg
3731    img_06025.jpg
1178    img_06375.jpg
2998    img_07193.jpg
2588    img_00738.jpg
2612    img_00899.jpg
2035    img_03127.jpg
2166    img_05413.jpg
103     img_01346.jpg
3667    img_02874.jpg
2268    img_01850.jpg
2284    img_03666.jpg
583     img_01652.jpg
3522    img_01521.jpg
1420    img_06136.jpg
3369    img_05832.jpg
2733    img_04963.jpg
2211    img_02134.jpg
369     img_06086.jpg
1117    img_00014.jpg
2949    img_01297.jpg
370     img_05762.jpg
3671    img_02970.jpg
111     img_06458.jpg
1897    img_06022.jpg
528     img_06773.jpg
2719    img_05803.jpg
3234    img_02893.jpg
348     img_00863.jpg
858     img_01858.jpg
            ...      
2020    img_01002.jpg
1603    img_01361.jpg
2514    img_01307.jpg
1356    img_02415.jpg
1515    img_00631.jpg
1434    img_03005.jpg
1990    img_02203.jpg
2474    img_03634.jpg
3617    img_06987.jpg
153     img_00641.jpg
3358    img_07892.jpg
2899    img_03062.jpg
1397    img_03662.jpg
1102    img_04330.jpg
3720    im

In [7]:
print y.shape

(3777, 8)


## Automated preprocessing: fish finder

In [8]:
# do stuff here


# Feature extraction

## Definition

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(input=base_model.input, output=base_model.get_layer('block4_pool').output)

def feature_generator(x, reshape=False):
    x = preprocess_input(x)
    pred = model.predict(x)
    if reshape:
        pred = pred.reshape(pred.shape[0],-1)
    return pred

## Generation

In [10]:
# Disabled by default, takes ~5min
feat = feature_generator(x)

KeyboardInterrupt: 

## Saving features

In [26]:
numpy.save('featurescropped.npy', feat)

## Loading features

In [11]:
feat = numpy.load('features.npy')

In [21]:
print feat.shape
x_train_feat = feat[:split_no,:,:,:]
x_train_feat.shape

x_test_feat = feat[split_no:,:,:,:]

(3777, 512, 14, 14)


## Top Model

In [24]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=(512, 14, 14)))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(8, activation='sigmoid'))
top_model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['binary_crossentropy'])
top_model = make_parallel(top_model, 8)
#top_model.fit(x_feat, y_train)

In [25]:
top_model.fit(x_train_feat, y_train, validation_data=(x_test_feat, y_test))
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
classes = model.predict_proba(x_test, batch_size=32)

# get probability table
proba_df = pd.DataFrame(classes)
proba_df.columns = image_df.groupby('fish_type').count().index # workaround to get column names quickly and in right order

print("Accuracy: %.2f%%" % (scores[1]*100))
print ' '
proba_df

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
model.predict()